<a href="https://colab.research.google.com/github/Vishwaak/Time-for-Machine/blob/master/Validation_and_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torchvision import datasets, transforms
import helper

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the training data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [0]:
from torch import nn, optim
import torch.nn.functional as F


In [0]:
#Network model.
class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784 , 256)
    self.fc2 = nn.Linear(256 , 128)
    self.fc3 = nn.Linear(128 , 64)
    self.fc4 = nn.Linear(64 , 10)
    
  def forward(self ,x):
    x = x.view(x.shape[0] , -1)
    
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.log_softmax(self.fc4(x) , dim =1)
    
    return x
     
    
   
  

In [26]:
model = Classifier()

images, labels = next(iter(testloader))
# Get the class probabilities
ps = torch.exp(model(images))
# Make sure the shape is appropriate, we should get 10 class probabilities for 64 examples
print(ps.shape)

torch.Size([64, 10])


In [27]:
top_p, top_class = ps.topk(1, dim=1)
# Look at the most likely classes for the first 10 examples
print(top_class[:10,:])

tensor([[4],
        [4],
        [8],
        [8],
        [8],
        [4],
        [4],
        [8],
        [4],
        [4]])


In [0]:
equals = top_class == labels.view(*top_class.shape)

In [30]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

Accuracy: 6.25%


In [33]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.003)

epochs = 30
steps = 0

train_losses, test_losses = [] , []
for e in range(epochs):
  running_loss = 0
  for images , labels in trainloader:
    
    optimizer.zero_grad()
    
    log_ps = model(images)
    loss = criterion(log_ps , labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    
  else:
      test_loss = 0
      accuracy = 0
      
      with torch.no_grad():
        for images , labels in testloader:
          log_ps = model(images)
          test_loss += criterion(log_ps , labels)
          
          ps = torch.exp(log_ps)
          top_p , top_class = ps.topk(1 , dim=1)
          equals = top_class == labels.view(*top_class.shape)
          accuracy += torch.mean(equals.type(torch.FloatTensor))
    
  train_losses.append(running_loss/len(trainloader))
  test_losses.append(test_loss/len(testloader))
    
  print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
    
    
    

Epoch: 1/30..  Training Loss: 0.513..  Test Loss: 0.455..  Test Accuracy: 0.829
Epoch: 2/30..  Training Loss: 0.391..  Test Loss: 0.389..  Test Accuracy: 0.858
Epoch: 3/30..  Training Loss: 0.352..  Test Loss: 0.393..  Test Accuracy: 0.861
Epoch: 4/30..  Training Loss: 0.332..  Test Loss: 0.381..  Test Accuracy: 0.868
Epoch: 5/30..  Training Loss: 0.316..  Test Loss: 0.396..  Test Accuracy: 0.865
Epoch: 6/30..  Training Loss: 0.304..  Test Loss: 0.382..  Test Accuracy: 0.866
Epoch: 7/30..  Training Loss: 0.291..  Test Loss: 0.386..  Test Accuracy: 0.867
Epoch: 8/30..  Training Loss: 0.281..  Test Loss: 0.380..  Test Accuracy: 0.872
Epoch: 9/30..  Training Loss: 0.272..  Test Loss: 0.359..  Test Accuracy: 0.875
Epoch: 10/30..  Training Loss: 0.270..  Test Loss: 0.368..  Test Accuracy: 0.881
Epoch: 11/30..  Training Loss: 0.258..  Test Loss: 0.358..  Test Accuracy: 0.878
Epoch: 12/30..  Training Loss: 0.254..  Test Loss: 0.388..  Test Accuracy: 0.867
Epoch: 13/30..  Training Loss: 0.244.